# Computing physical statistics
_@Lutecity April 2023_

**Description**

This notebook computes the metabolic power.

This notebook also processes data provided by Second Spectrum to match the dataset format used in Laurie Shaw package. The dataset is exported and used in the app.

The inspiration and large pieces of code come from the work of Sudarshan Gopaladesikan [Github: Sportsdatascience](https://github.com/slbenfica1079/sportsdatascience) and Laurie Shaw [Tutorial on Friends of Tracking](https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking)

**Summary**

1. [Import the data](#data)
2. [Metabolic power](#metabolic_power)
    1. [Calculate player velocities](#velocities)
    2. [Number of Accelerations and Decelerations](#nb_acc_dec)
    3. [Calculate metabolic cost and metabolic power](#metabolic_cost)

    4. [Find the spot where substitution should happen](#threshold)

## Import the data <a id="data"></a>
#### Librairies

In [5]:
import pandas as pd 
import numpy as np
import os
import seaborn
from sklearn import linear_model
import matplotlib.pyplot as plt
import scipy as sp
import ruptures as rpt
import math

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('max_colwidth', 400)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

# Plotly
import plotly.graph_objs as go
from floodlight.io.secondspectrum import read_position_data_jsonl, read_teamsheets_from_meta_json

Laurie Shaw package:

In [2]:
import LaurieOnTracking_package.Metrica_IO as mio
import LaurieOnTracking_package.Metrica_Velocities as mvel
import LaurieOnTracking_package.Metrica_Viz as mviz

#### Data

In [3]:
tracking_home = pd.read_csv('Output/tracking_home.csv',sep=";", encoding = "utf-8-sig")
tracking_away = pd.read_csv('Output/tracking_away.csv',sep=";", encoding = "utf-8-sig")

## Metabolic power <a id="metabolic_power"></a>

### Calculate player velocities <a id="velocities"></a> 

In [4]:
tracking_home = mvel.calc_player_velocities(tracking_home,smoothing=True)
tracking_away = mvel.calc_player_velocities(tracking_away,smoothing=True)

### Number of Accelerations and Decelerations <a id="nb_acc_dec"></a> 

In [5]:
def calculate_accelerations(df_tracking, teamname) : 
    player_columns = [c.split('_')[1] for c in df_tracking.columns if c[-2:].lower()=='_x' and c[:4] in ['Home','Away']]
    maxacc = 6
    for player in player_columns:
        df_tracking[teamname+ '_' + player + '_Acc'] = df_tracking[teamname+ '_' + player + '_speed'].diff() / df_tracking['Time [s]'].diff()
        df_tracking[teamname+ '_' + player + '_Acc'].loc[np.absolute(df_tracking[teamname+ '_' + player + '_Acc']) > maxacc] = np.nan
    return(df_tracking)

In [6]:
tracking_home = calculate_accelerations(tracking_home, 'Home')
tracking_away = calculate_accelerations(tracking_away, 'Away')

### Calculate metabolic cost and metabolic power <a id="metabolic_cost"></a> 

In [7]:
# Introduce concept of metabolic power and SPI
def split_at(s, c, n):
    words = s.split(c)
    return c.join(words[:n]), c.join(words[n:])


def metabolic_cost(acc):
    if acc > 0:
        # es = acc / 9.80665
        # em = (es ** 2 + 1) ** 0.5
        cost = 0.102 * ((acc ** 2 + 96.2) ** 0.5) * (4.03 * acc + 3.6 * np.exp(-0.408 * acc))
    elif acc < 0:
        # es = acc / 9.80665
        # em = (es ** 2 + 1) ** 0.5
        cost = 0.102 * ((acc ** 2 + 96.2) ** 0.5) * (-0.85 * acc + 3.6 * np.exp(1.33 * acc))
    else:
        cost = 0
    return cost

In [8]:
def compute_metabolic_power(df_tracking, teamname):
    player_columns = [c.split('_')[1] for c in df_tracking.columns if c[-2:].lower()=='_x' and c[:4] in ['Home','Away']]

    for player in player_columns:
        df_tracking[teamname+ '_' + player + '_Metabolic_cost'] = df_tracking[teamname+ '_' + player + '_Acc'].apply(lambda x: metabolic_cost(x))
        df_tracking[teamname+ '_' + player + '_Metabolic_cost'] = df_tracking[teamname+ '_' + player + '_Metabolic_cost']  *df_tracking[teamname+ '_' + player + '_speed']
        df_tracking[teamname+ '_' + player + '_Metabolic_power'] = df_tracking[teamname+ '_' + player + '_Metabolic_cost'].rolling(7500,min_periods=1).apply(lambda x : np.nansum(x))

    return(df_tracking)

In [9]:
tracking_home = compute_metabolic_power(tracking_home, 'Home')
tracking_away = compute_metabolic_power(tracking_away, 'Away')

We only keep one row per seconds. We compute the mean of the metabolic power per second

In [ ]:
def compute_mean_of_metabolic_power_per_second(df_tracking) : 
    df_tracking['Time_sec'] = df_tracking['Time [s]'].round()
    df_tracking = df_tracking[['Time_sec', 'Period'] + df_tracking.columns[df_tracking.columns.str.contains("Metabolic_power")].tolist()]
    df_tracking = df_tracking.groupby(['Period','Time_sec']).mean().reset_index()
    df_tracking['time'] = df_tracking['Time_sec'].apply(lambda x: math.ceil(x /60))
    df_tracking['official_clock'] = np.where(df_tracking['Period'] == 1,
                                np.where(df_tracking['time'] > 45,
                                        df_tracking['time'].apply(lambda x: str(45) + "'+" + str(x - 45)),
                                        df_tracking['time'].astype(str)
                                ), 
                                np.where(df_tracking['time'] > 90,
                                        df_tracking['time'].apply(lambda x: str(90) + "'+" + str(x - 90)),
                                        df_tracking['time'].astype(str)
                                ),
                            )
    return(df_tracking)

In [ ]:
tracking_home_sub = compute_mean_of_metabolic_power_per_second(tracking_home)
tracking_away_sub = compute_mean_of_metabolic_power_per_second(tracking_away)

C:\Users\mbaconnais\AppData\Local\Temp\ipykernel_11776\421376496.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tracking['Time_sec'] = df_tracking['Time [s]'].round()
C:\Users\mbaconnais\AppData\Local\Temp\ipykernel_11776\421376496.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_tracking['Time_sec'] = df_tracking['Time [s]'].round()


##### Export

In [ ]:
# tracking_home_sub.to_csv('Output/metabolic_power_home.csv',encoding='utf-8-sig',sep = ";", index=False, header=True)
# tracking_away_sub.to_csv('Output/metabolic_power_away.csv',encoding='utf-8-sig',sep = ";", index=False, header=True)

In [6]:
# tracking_home_sub = pd.read_csv('Output/metabolic_power_home.csv',sep=";", encoding = "utf-8-sig")
# tracking_away_sub = pd.read_csv('Output/metabolic_power_away.csv',sep=";", encoding = "utf-8-sig")

### Find the spot where substitution should happen <a id="threshold"></a> 

In [ ]:
def find_fatigue_threshold(metabolic_power) : 
    '''
    This function find the spot where substitution should happen.
    '''
    signal = np.array(metabolic_power).reshape((len(metabolic_power),1))
    algo = rpt.Binseg(model="l2").fit(signal) ##potentially finding spot where substitution should happen
    result = algo.predict(n_bkps=1) #big_seg
    result = [x + 3000 for x in result]
    return(result)

Let's visualize the metabolic power on a player

In [10]:
df_tracking = tracking_home_sub
metabolic_power = df_tracking[df_tracking['Time_sec'] > 3000]['Home_9_Metabolic_power']
result = find_fatigue_threshold(metabolic_power)

In [9]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = df_tracking['Time_sec'],
        y = df_tracking['Home_9_Metabolic_power'],
        fillcolor = "#8bcbfc"    
    )
)

fig.add_vrect(
    x0=result[0], 
    x1=result[1], 
    line_width=0, 
    fillcolor="red", 
    opacity=0.2
)

# Define x tick as official time
df_official_clock = df_tracking[['Time_sec','official_clock']].drop_duplicates()
df_official_clock['official_mod'] = df_tracking['official_clock'].apply(lambda x: int(x.split("'+")[-1]))
df_official_clock = df_official_clock[df_official_clock['official_mod'] % 5 == 0].drop_duplicates(subset = 'official_clock')
x_tick_val = df_official_clock['Time_sec'].tolist()
x_tick_labels = df_official_clock['official_clock'].tolist()

fig.update_layout(
    xaxis = dict(
            title = 'Time',
            tickmode = 'array',
            tickvals = x_tick_val,
            ticktext = x_tick_labels
        ),
    yaxis = {
        'title' : 'Metabolic power'
    },
    template="plotly_white",
    legend = dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    font=dict(
        size=9,
    ),
    margin=dict(
        l=0,
        r=10,
        b=0,
        t=0
    ),
    hoverlabel=dict(
        font_size=11,
    ),
    height = 275,
)
fig.show()
